In [1]:
import numpy as np
import os
import pickle
from karabo.imaging.imager_oskar import OskarDirtyImager
from karabo.imaging import imager_wsclean
from karabo.simulation.interferometer import InterferometerSimulation
from karabo.simulation.observation import Observation
from karabo.simulation.sky_model import SkyModel
from karabo.simulation.telescope import Telescope
from karabo.simulation.visibility import Visibility
from karabo.imaging.imager_oskar import OskarDirtyImager, OskarDirtyImagerConfig
from karabo.simulator_backend import SimulatorBackend
from astropy.io import fits
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib as mpl
#mpl.use('Tkagg')
from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    return

In [7]:
def get_baselines(telescope_path):
    print(telescope_path)
    layout=np.loadtxt(telescope_path+'/layout.txt')
    nant=len(layout)
    nb=int(nant*(nant-1)*0.5)
    base_length=[0]*nb
    k=0
    for i in range(nant):
        for j in range(i,nant):
            if(i!=j):
                base_length[k] = np.sqrt((layout[i][0]-layout[j][0])**2 + (layout[i][1]-layout[j][1])**2)
                k=k+1
    base_length=np.array(base_length)
    telname = telescope_path.split('/')[-1].split('.')[0]
    return base_length,nant,nb,telname

In [ ]:
meerkat_path = '/data/rohit/skao_repo/meerkat.tm'
ska1mid_path = '/data/rohit/skao_repo/ska1mid.tm'
skamid_AAs_path = '/data/rohit/skao_repo/ska-mid-AAstar.tm'
skamid_AA2_path = '/data/rohit/skao_repo/ska-mid-AA2.tm'
skamid_AA1_path = '/data/rohit/skao_repo/ska-mid-AA1.tm'
#---------------
mwa_ph1_path = '/data/rohit/skao_repo/mwa.phase1.tm'
ska1low_path = '/data/rohit/skao_repo/ska1low.tm'
skalow_AAs_path = '/data/rohit/skao_repo/ska-low-AAstar.tm'
skalow_AA2_path = '/data/rohit/skao_repo/ska-low-AA2.tm'
skalow_AA1_path = '/data/rohit/skao_repo/ska-low-AA1.tm'

telescope_mid_list = [ska1mid_path, skamid_AAs_path, skamid_AA2_path, skamid_AA1_path, meerkat_path]
telescope_low_list = [ska1low_path, skalow_AAs_path, skalow_AA2_path, skalow_AA1_path, mwa_ph1_path]
telescope_list = telescope_low_list + telescope_mid_list

freq_low_arr = np.arange(50,350) # in MHz
freq_mid_arr = np.arange(350,15000) # in MHz

base_length = [0]*len(telescope_list)
nant = [0]*len(telescope_list)
nb = [0]*len(telescope_list)
nbmax = [0]*len(telescope_list)
telname = [0]*len(telescope_list)
beamsize_arr = [0]*len(telescope_list)
beamsize_arr_arcsec = [0]*len(telescope_list)
i=0
for tel in telescope_list:
    base_length[i],nant[i],nb[i],telname[i] = get_baselines(telescope_list[i])
    nbmax[i] = base_length[i].max()
    if(i<5):
        freq_list = freq_mid_arr
    else:
        freq_list = freq_low_arr
    #print(telname[i], freq_list)
    beamsize_arr[i] = 3.e8/base_length[i].max()/(freq_list*1.e6)
    beamsize_arr_arcsec[i] = beamsize_arr[i]*180/np.pi*3600
    i=i+1
#print('Maximum Baseline',base_length.max())
#----------------------------


/data/rohit/skao_repo/ska1low.tm
ska1low [  350   351   352 ... 14997 14998 14999]
/data/rohit/skao_repo/ska-low-AAstar.tm
ska-low-AAstar [  350   351   352 ... 14997 14998 14999]
/data/rohit/skao_repo/ska-low-AA2.tm
ska-low-AA2 [  350   351   352 ... 14997 14998 14999]
/data/rohit/skao_repo/ska-low-AA1.tm
ska-low-AA1 [  350   351   352 ... 14997 14998 14999]
/data/rohit/skao_repo/mwa.phase1.tm
mwa [  350   351   352 ... 14997 14998 14999]
/data/rohit/skao_repo/ska1mid.tm
ska1mid [ 50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157
 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175
 176

In [13]:

f,ax=plt.subplots(1,1)
ax.hist(nb[0],bins=100)
ax.set_xlabel('Frequency (MHz)')
ax.set_ylabel('Max. Resolution (arcsec)')
ax.set_title('SKA-mid')
plt.show()

/tmp/ipykernel_2200842/1708583086.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
